In [1]:
# Imports

import argparse
import json
import logging
import os
import random
import sys
from dataclasses import dataclass
from typing import Tuple

import numpy as np
import pandas as pd
import torch
from torch import Tensor

from reagent.ope.estimators.contextual_bandits_estimators import (
    Action,
    ActionDistribution,
    ActionRewards,
    BanditsEstimatorInput,
    BanditsModel,
    DMEstimator,
    DoublyRobustEstimator,
    IPSEstimator,
    LogSample,
)
from reagent.ope.estimators.types import ActionSpace, Policy
from reagent.ope.trainers.linear_trainers import (
    LogisticRegressionTrainer,
    SGDClassifierTrainer,
    TrainingData,
    DecisionTreeTrainer,
    LinearTrainer
)
from reagent.ope.test.multiclass_bandits import (
    MultiClassDataRow,
    UCIMultiClassDataset,
    MultiClassContext,
    MultiClassModel,
    MultiClassPolicy,
    evaluate_all
)
from reagent.ope.utils import RunningAverage

import matplotlib
import matplotlib.pyplot as plt

## Configuration Settings

Edit the experiments list with the names of UCI datasets given in reagent/test/data to produce results for each dataset. 

In [2]:
# Configuration

DEFAULT_ITERATIONS = 500
TEST_ROOT_PATH = '..'
UCI_DATASET_CONFIGS = os.path.join(TEST_ROOT_PATH, 'configs')
MAX_METRIC_NAME_LENGTH = 20
experiments = ["ecoli", "letter_recog", "pendigits", "optdigits", "satimage"]

experiment_params = []
for exp in experiments:
    with open(os.path.join(UCI_DATASET_CONFIGS, exp + '_config.json'), "r") as f:
        params = json.load(f)
        if "dataset" in params:
            if "file" in params["dataset"]:
                params["dataset"]["file"] = os.path.join(TEST_ROOT_PATH, params["dataset"]["file"])
        experiment_params.append({"name": exp, "params": params})     

## Run an experiment

We load the given dataset, and create trainers (which will be used for generating the policies for the logger and target). To try different trainers, modify the `log_trainer` and `tgt_trainer` variables with different `LinearTrainer`s. 

Note that DM's performance is highly dependent on the reward model. To try different reward models, modify the trainer passed into `DMEstimator` and `DoublyRobustEstimator` with different `LinearTrainer`s. 

In [3]:
# Experiment(s)
def run_experiment(params): 
    random.seed(1234)
    np.random.seed(1234)
    torch.random.manual_seed(1234)

    dataset = UCIMultiClassDataset(params["dataset"])
    log_trainer = LogisticRegressionTrainer()
    log_epsilon = 0.1
    tgt_trainer = SGDClassifierTrainer()
    tgt_epsilon = 0.1
    experiments = [
        (
            (
                DMEstimator(LogisticRegressionTrainer()),
                IPSEstimator(),
                DoublyRobustEstimator(LogisticRegressionTrainer()),
            ),
            1000,
        )
        for _ in range(100)
    ]
    results = evaluate_all(
        experiments, dataset, log_trainer, log_epsilon, tgt_trainer, tgt_epsilon, 0
    )
    return results


## Result Generation

For each UCI dataset, we generate a logging and target policy, create a simulated dataset using the logging policy, and evaluate the target policy using DM, IPS, and DR. The bias, rmse, and variance against the ground truth is plotted for each dataset. 


For the settings with the logging policy trained with a `LogisticRegressionTrainer`, the target policy with a `SGDClassifierTrainer`, and the reward model for DM and DR trained with a `LogisticRegressionTrainer`, a sample result gives:


![alt text](img/bias.png "Bias")![alt text](img/variance.png "Bias")![alt text](img/rmse.png "Bias")

In [ ]:
# Generate Bar Charts, a la https://arxiv.org/pdf/1511.03722.pdf

def create_and_show_chart(labels, results, title):
    # Width of each bar
    width = 0.25

    metrics = list(results.keys())
    
    # Set position of bar on X axis
    barpos = [np.arange(len(results[metrics[0]]))]
    for m in range(len(metrics)-1):
        barpos.append([x + width for x in barpos[-1]])
        
    fig, ax = plt.subplots()
    for metric, barpositions in zip(metrics, barpos):
        ax.bar(barpositions, results[metric], width, label=metric[:MAX_METRIC_NAME_LENGTH])

    ax.set_ylabel(title)
    plt.xticks([r + width for r in range(len(labels))], labels)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

labels = []

bias_result_mapping = {}
var_result_mapping = {}
rmse_result_mapping = {}

for params in experiment_params:
    print("Running experiment " + params["name"])
    exp_results = run_experiment(params["params"])
    labels.append(params["name"])
    
    for estimator_name, result in exp_results.items():
        _, _, _, tgt_gt, _, _ = result.report()
        if not estimator_name in bias_result_mapping:
            bias_result_mapping[estimator_name] = []
        if not estimator_name in var_result_mapping:
            var_result_mapping[estimator_name] = []
        if not estimator_name in rmse_result_mapping:
            rmse_result_mapping[estimator_name] = []
            
        bias_result_mapping[estimator_name].append(tgt_gt.bias.cpu().numpy())
        var_result_mapping[estimator_name].append(tgt_gt.variance.cpu().numpy())
        rmse_result_mapping[estimator_name].append(tgt_gt.rmse.cpu().numpy())

create_and_show_chart(labels, bias_result_mapping, 'Bias')
create_and_show_chart(labels, var_result_mapping, 'RMSE')
create_and_show_chart(labels, rmse_result_mapping, 'Variance')

Running experiment ecoli
